### Generate coordinates

In [1]:
import pandas as pd
import numpy as np
import random
import geopandas as gpd
from random import shuffle
from shapely.geometry import Polygon, Point

/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# data that contain polygons of all zip codes in Germany
plz_shape_df = gpd.read_file('/plz-gebiete.shp', dtype={'plz': str})
plz_shape_df.head()

,plz,note,geometry
0,52538,"52538 Gangelt, Selfkant","POLYGON ((5.86632 51.05110, 5.86692 51.05124, ..."
1,47559,47559 Kranenburg,"POLYGON ((5.94504 51.82354, 5.94580 51.82409, ..."
2,52525,"52525 Waldfeucht, Heinsberg","POLYGON ((5.96811 51.05556, 5.96951 51.05660, ..."
3,52074,52074 Aachen,"POLYGON ((5.97486 50.79804, 5.97495 50.79809, ..."
4,52531,52531 Ãbach-Palenberg,"POLYGON ((6.01507 50.94788, 6.03854 50.93561, ..."


In [3]:
# filter a some from Munich
mu_zip_codes = ["80331"]
plz_shape_df = plz_shape_df[plz_shape_df.plz.isin(mu_zip_codes)]
plz_shape_df

,plz,note,geometry
6707,80331,80331 MÃ¼nchen,"POLYGON ((11.56320 48.13661, 11.56321 48.13696..."


In [4]:
lat = np.arange(48.137023, 48.137323, 0.0000005).tolist()
lon = np.arange(11.574993, 11.575993, 0.0000005).tolist()
shuffle(lat)
shuffle(lon)

In [5]:
# generate random point inside polygon
def get_random_point_in_polygon(poly):
     minx, miny, maxx, maxy = poly.bounds
     while True:
         p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
         if poly.contains(p):
             return p

# iterate over all given polygons          
for polygon in list(plz_shape_df.geometry):
    # generate n random points
    for i in range(0, 2000):
        # generate point
        shapely_point = get_random_point_in_polygon(polygon)
        # get point coordinates
        coords = shapely_point.coords.xy
        # get lat
        latitude = coords[1][0]
        # get lon
        longtitude = coords[0][0]
        # add to coords lists
        lat.append(latitude)
        lon.append(longtitude)

In [6]:
latlon = list(zip(lat, lon))

### Save df to firebase 

In [15]:
import firebase_admin
from firebase_admin import db, storage, credentials

In [16]:
# setup connection
if not firebase_admin._apps:
    cred = credentials.Certificate('/Users/kateshatik/Documents/LMU/Hackathon/firebase_apikey.json') 
    default_app = firebase_admin.initialize_app(cred, {'databaseURL': "https://firebase_db.firebaseio.com"})

In [ ]:
# Get a database reference to our posts
ref = firebase_admin.db.reference("users")

In [25]:
root = db.reference()
# Add a new user under /users.
for coord in latlon:
    new_user = root.child('users').push({
        'Latitude' : coord[0],
        'Longitude' : coord[1]
    })

In [26]:
# Read the data at the posts reference (this is a blocking operation)
print(ref.get())

{'-MIE0wiYRMhXTpl4fRRe': {'Latitude': 48.137321, 'Longitude': 11.575555}, '-MIE1aGP6aahUm3Il7mM': {'Latitude': 48.137321, 'Longitude': 11.575555}, '-MIE2LxILKungei8KjG3': {'Latitude': 48.13727899999935, 'Longitude': 11.575554500000576}, '-MIE2M0GURSdSoQD5d2u': {'Latitude': 48.13732099999925, 'Longitude': 11.575155500000166}, '-MIE2M4VUEJf0dhY5Prl': {'Latitude': 48.13719699999956, 'Longitude': 11.575191500000203}, '-MIE2M8-cQ5HezXOZX7O': {'Latitude': 48.137036499999965, 'Longitude': 11.575004500000011}, '-MIE2MBcfKTyCWOj-cCe': {'Latitude': 48.13731099999927, 'Longitude': 11.575040500000048}, '-MIE2MEoPfj1lQI0Sppa': {'Latitude': 48.13731549999926, 'Longitude': 11.575493500000514}, '-MIE2MITKAM8pz11L4Ut': {'Latitude': 48.13724949999943, 'Longitude': 11.575428500000447}, '-MIE2MLx_ZS2z5AjKmmH': {'Latitude': 48.1371409999997, 'Longitude': 11.575138500000149}, '-MIE2MPsQFtX5iDz2oCw': {'Latitude': 48.13711699999976, 'Longitude': 11.575552500000574}, '-MIE2MU99bKQ7_UgJSM9': {'Latitude': 48.137